In [2]:
from capture_image import capture_image
import sys
import os
import warnings
import cv2
import numpy as np

# add parent folder to sys.path
sys.path.insert(0, os.path.abspath(".."))

# ignore DeprecationWarning messages
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ignore RuntimeWarning messages
warnings.filterwarnings("ignore", category=RuntimeWarning)

from volume_estimation.src.models_no_input_vol.predict_full_pipeline import predict
from segmentation_and_depth.src.models.predict_model import predict as predict_seg_depth

In [3]:
path = capture_image()
print("Path: ", path)
predict(path, save_segmentation=True, save_depth=True, predict_volume=False)

Image saved
Path:  captured_images/20230607_164059/image.png


In [7]:
# load saved segmentation map as npy file
segmentation_map_npy = np.load(path[:-4] + "_VesselOpeningMask.npy")
print("Segmentation map shape: ", segmentation_map_npy.shape)

Segmentation map shape:  (1, 480, 640)


In [8]:
# value count of segmentation map
unique, counts = np.unique(segmentation_map_npy, return_counts=True)
print("Value count of segmentation map: ", dict(zip(unique, counts)))


Value count of segmentation map:  {0: 305333, 1: 1867}


In [4]:
model_path = "../segmentation_and_depth/models/segmentation_depth_model.torch"
predict_seg_depth(model_path, path)

FileNotFoundError: [Errno 2] No such file or directory: 'output/model.txt'